In [29]:
"""
Calculates the amount of anomalous data points in the NZERTF data
"""
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest

import DataRetriever as dr

retriever = dr.DataRetriever()

year_two = retriever.get_data("All-Subsystems-minute-Year2.pkl")
year_two.drop(['Timestamp', 'TimeStamp_Count', 'DayOfWeek'], axis=1, inplace=True)
year_two.fillna(0, inplace=True)

year_two = pd.DataFrame(np.float32(year_two))

In [32]:
year_two.values

array([[ 0.       ,  4.9947147,  0.       , ..., 72.102    , 74.655    ,
         0.       ],
       [ 0.       ,  5.0467916,  0.       , ..., 72.096    , 74.701    ,
         0.       ],
       [ 0.       ,  5.1294928,  0.       , ..., 72.077    , 74.759    ,
         0.       ],
       ...,
       [ 1.763535 ,  4.548853 ,  0.       , ..., 67.644    , 68.549    ,
         0.       ],
       [ 1.763535 ,  4.557287 ,  0.       , ..., 67.651    , 68.552    ,
         0.       ],
       [ 1.763535 ,  4.589046 ,  0.       , ..., 67.652    , 68.56     ,
         0.       ]], dtype=float32)

In [31]:
iso_forest = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', max_features=1.0,
                             bootstrap=False, n_jobs=1, random_state=1, warm_start=False)

iso_forest.fit(year_two.values)
predictions = iso_forest.predict(year_two.values)
amount_anomalous = len(np.where(predictions < 0)[0])
amount_not_anomalous = len(np.where(predictions > 0)[0])

proportion_anomalous = amount_anomalous / len(year_two)
proportion_not_anomalous = amount_not_anomalous / len(year_two)
print(f'Proportion anomalous: {amount_anomalous}/{len(year_two)} = {round(proportion_anomalous * 100, 2)}%')
print(f'Proportion not anomalous: {amount_not_anomalous}/{len(year_two)} = {round(proportion_not_anomalous * 100, 2)}%')

anomalous_indeces = np.where(predictions == -1)[0].tolist()
print(year_two.iloc[anomalous_indeces])

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
np.where(np.isinf(year_two))